In [1]:
import pandas as pd
import joblib

model = joblib.load("../../Models/unilorin_admission_model.pkl")
print("Model loaded successfully!")


Model loaded successfully!


In [ ]:
olevel_scale = {'A1': 4.0, 'B2': 3.6, 'B3': 3.2, 'C4': 2.8, 'C5': 2.4, 'C6': 2.0}

def calculate_olevel_avg(grades):
    """Compute average O'Level grade points (UNILORIN scale)."""
    points = [olevel_scale[g] for g in grades]
    return round(sum(points) / len(points), 2)

def calculate_aggregate(utme, post_utme, olevel_avg):
    """Compute UNILORIN weighted aggregate score: 50% UTME + 30% Post-UTME + 20% O'Level."""
    utme_score = (utme / 400) * 50
    post_utme_score = (post_utme / 30) * 30
    olevel_score = olevel_avg * 5  # scaled to 20%
    total = utme_score + post_utme_score + olevel_score
    return round(total, 2)

# -----------------------------
# 3️⃣ Applicant Data (Modify This Section)
# -----------------------------
applicant = {
    "Faculty": "Management Sciences",
    "Department": "Business Administration",
    "UTME_Score": 265,
    "Post_UTME_Score": 24.5,  # out of 30
    "Olevel_Grades": ["B3", "B2", "C4", "A1", "C5"],
    "Olevel_Valid": 1,  # 1 = passed, 0 = failed
}

# -----------------------------
# 4️⃣ Process Input
# -----------------------------
# Compute average O'Level points
olevel_avg_points = calculate_olevel_avg(applicant["Olevel_Grades"])

# Compute aggregate score
aggregate_score = calculate_aggregate(
    applicant["UTME_Score"],
    applicant["Post_UTME_Score"],
    olevel_avg_points
)

# Assemble applicant DataFrame
input_df = pd.DataFrame([{
    "Faculty": applicant["Faculty"],
    "Department": applicant["Department"],
    "UTME_Score": applicant["UTME_Score"],
    "Post_UTME_Score": applicant["Post_UTME_Score"],
    "Aggregate_Score": aggregate_score,
    "Olevel_Valid": applicant["Olevel_Valid"],
    "olevel_avg_points": olevel_avg_points
}])

print("\n📋 Applicant Data:")
print(input_df)

# -----------------------------
# 5️⃣ Predict Admission Probability
# -----------------------------
probability = model.predict_proba(input_df)[0][1]  # Probability of being admitted
chance_percent = probability * 100

# Threshold-based decision
if probability >= 0.7:
    chance_label = "High"
elif 0.5 <= probability < 0.7:
    chance_label = "Moderate"
else:
    chance_label = "Low"

# -----------------------------
# 6️⃣ Display Results
# -----------------------------
print("\n🎯 Prediction Result:")
print(f"📊 Aggregate Score: {aggregate_score}")
print(f"📘 O'Level Average Points: {olevel_avg_points}")
print(f"💡 Admission Probability: {chance_percent:.2f}% ({chance_label} Chance)")

if chance_label == "High":
    print("✅ Likely to be admitted into UNILORIN!")
elif chance_label == "Moderate":
    print("⚖️ Fair chance — performance is competitive, but not guaranteed.")
else:
    print("❌ Low chance of admission — consider changing course or improving scores.")



📋 Applicant Data:
               Faculty               Department  UTME_Score  Post_UTME_Score  \
0  Management Sciences  Business Administration         180             14.5   

   Aggregate_Score  Olevel_Valid  olevel_avg_points  
0             53.0             1                3.2  

🎯 Prediction Result:
📊 Aggregate Score: 53.0
📘 O'Level Average Points: 3.2
💡 Admission Probability: 61.20% (Moderate Chance)
⚖️ Fair chance — performance is competitive, but not guaranteed.
